In [271]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [272]:
# get data files
#!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

#!unzip book-crossings.zip

In [273]:
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [274]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})
df_ratings.count()

user      1149780
isbn      1149780
rating    1149780
dtype: int64

In [275]:
user_value_counts = df_ratings['user'].value_counts()
book_value_counts = df_ratings['isbn'].value_counts()

In [276]:
user_filtered_values = user_value_counts[user_value_counts >= 200]

df_ratings = df_ratings[df_ratings['user'].isin(user_filtered_values.index)]

In [277]:
book_filtered_values = book_value_counts[book_value_counts >= 100]

df_ratings = df_ratings[df_ratings['isbn'].isin(book_filtered_values.index)]

In [278]:
df_ratings.count()

user      49781
isbn      49781
rating    49781
dtype: int64

In [279]:
merged_df = pd.merge(df_ratings, df_books, on='isbn', how='inner')

In [280]:
merged_df = merged_df.drop_duplicates(['title', 'user'])

In [281]:
df_pivot = merged_df.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)

In [282]:
model = NearestNeighbors(n_neighbors=5, metric='cosine', algorithm='auto', p=2)

In [283]:
nbrs = model.fit(df_pivot)

In [284]:
# function to return recommended books - this will be tested
def get_recommends(book=""):

  number_of_books = df_pivot.shape[0]
  query_vector = np.array([df_pivot.loc["Where the Heart Is (Oprah's Book Club (Paperback))"].values])
  distances, indices = nbrs.kneighbors(query_vector, number_of_books)

  recommended_books = []
    
  for i in range(1, 6):
    recomendation_index = indices[0][i]
    recomendation_title = df_pivot.index[recomendation_index]
    recomendation_distance = distances[0][i]
    recommended_books.append([recomendation_title, recomendation_distance])

  recommended_books.reverse()
  return [book, recommended_books]

In [285]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
